In [51]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import dask_xgboost
import pandas as pd
import xgboost as xgb

ModuleNotFoundError: No module named 'dask_xgboost'

In [2]:
xgb.__version__

'1.5.0'

In [49]:
def main(client):
    # We use HIGGS as the dataset for demonstration. To obtain it, download:
    # https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz
    fname = '../input/HIGGS.csv'
    colnames = ['label'] + ['feature-%02d' % i for i in range(1, 29)]
    # By default dask dataframe uses pandas as data handling backend
    dask_df = dd.read_csv(fname, low_memory=False)
    y = dask_df['label']
    X = dask_df[dask_df.columns.difference(['label'])]
    # DaskDMatrix acts like normal DMatrix, works as a proxy for local
    # DMatrix scattered around the workers.
    dtrain = xgb.dask.DaskDMatrix(client, X, y)
    # Use train method from xgboost.dask instead of xgboost.  This
    # distributed version of train returns a dictionary containing the
    # resulting booster and evaluation history obtained from
    # evaluation metrics.
    output = xgb.dask.train(client,
                            # Use GPU training algorithm
                            {'verbosity':1, 'tree_method': 'gpu_hist', 
                             'objective':'binary:logistic'},
                            dtrain,
                            num_boost_round=10)
    booster = output['booster']  # booster is the trained model
    #booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, dtrain)
    history = output['history']  # A dictionary containing evaluation results
    # Save the model to file
    booster.save_model('xgboost-model')

    print('Some predictions:', predictions)

In [50]:
if __name__ == '__main__':
    # `LocalCUDACluster` is used for assigning GPU to XGBoost 
    # processes. Here `n_workers` represents the number of GPUs 
    # since we use one GPU per worker process.
    with LocalCUDACluster(n_workers=2) as cluster:
        with Client(cluster) as client:
            main(client)

distributed.preloading - INFO - Import preload module: dask_cuda.initialize
distributed.preloading - INFO - Import preload module: dask_cuda.initialize
[14:56:34] task [xgboost.dask]:tcp://192.168.1.200:38769 got new rank 0
[14:56:34] task [xgboost.dask]:tcp://192.168.1.200:39665 got new rank 1
[14:56:40] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:56:40] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Some predictions: dask.array<concatenate, shape=(11000000,), dtype=float32, chunksize=(126462,), chunktype=numpy.ndarray>


distributed.worker - ERROR - Worker stream died during communication: tcp://192.168.1.200:38769
Traceback (most recent call last):
  File "/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker.py", line 2387, in gather_dep
    response = await get_data_from_worker(
  File "/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker.py", line 3759, in get_data_from_worker
    return await retry_operation(_get_data, operation="get_data_from_worker")
  File "/home/tunguz/.local/lib/python3.8/site-packages/distributed/utils_comm.py", line 385, in retry_operation
    return await retry(
  File "/home/tunguz/.local/lib/python3.8/site-packages/distributed/utils_comm.py", line 370, in retry
    return await coro()
  File "/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker.py", line 3754, in _get_data
    await comm.write("OK")
  File "/home/tunguz/.local/lib/python3.8/site-packages/distributed/comm/tcp.py", line 241, in write
    raise CommClosedError()
dist

In [5]:
y = pd_df['label']
X = pd_df[pd_df.columns.difference(['label'])]

In [6]:
dtrain = xgb.dask.DaskDMatrix(client, X, y)

NameError: name 'client' is not defined

In [13]:
fname = '../input/HIGGS.csv'
colnames = ['label'] + ['feature-%02d' % i for i in range(1, 29)]
# By default dask dataframe uses pandas as data handling backend
dask_df = dd.read_csv(fname, low_memory=False)

In [14]:
dask_df.head()

,label,feature-01,feature-02,feature-03,feature-04,feature-05,feature-06,feature-07,feature-08,feature-09,...,feature-19,feature-20,feature-21,feature-22,feature-23,feature-24,feature-25,feature-26,feature-27,feature-28
0,1.0,0.869293,-0.635082,0.225690,0.327470,-0.689993,0.754202,-0.248573,-1.092064,0.000000,...,-0.010455,-0.045767,3.101961,1.353760,0.979563,0.978076,0.920005,0.721657,0.988751,0.876678
1,1.0,0.907542,0.329147,0.359412,1.497970,-0.313010,1.095531,-0.557525,-1.588230,2.173076,...,-1.138930,-0.000819,0.000000,0.302220,0.833048,0.985700,0.978098,0.779732,0.992356,0.798343
2,1.0,0.798835,1.470639,-1.635975,0.453773,0.425629,1.104875,1.282322,1.381664,0.000000,...,1.128848,0.900461,0.000000,0.909753,1.108330,0.985692,0.951331,0.803252,0.865924,0.780118
3,0.0,1.344385,-0.876626,0.935913,1.992050,0.882454,1.786066,-1.646778,-0.942383,0.000000,...,-0.678379,-1.360356,0.000000,0.946652,1.028704,0.998656,0.728281,0.869200,1.026736,0.957904
4,1.0,1.105009,0.321356,1.522401,0.882808,-1.205349,0.681466,-1.070464,-0.921871,0.000000,...,-0.373566,0.113041,0.000000,0.755856,1.361057,0.986610,0.838085,1.133295,0.872245,0.808487


In [12]:
pd_df = pd.read_csv(fname, low_memory=False)
pd_df.head()

,label,feature-01,feature-02,feature-03,feature-04,feature-05,feature-06,feature-07,feature-08,feature-09,...,feature-19,feature-20,feature-21,feature-22,feature-23,feature-24,feature-25,feature-26,feature-27,feature-28
0,1.0,0.869293,-0.635082,0.225690,0.327470,-0.689993,0.754202,-0.248573,-1.092064,0.000000,...,-0.010455,-0.045767,3.101961,1.353760,0.979563,0.978076,0.920005,0.721657,0.988751,0.876678
1,1.0,0.907542,0.329147,0.359412,1.497970,-0.313010,1.095531,-0.557525,-1.588230,2.173076,...,-1.138930,-0.000819,0.000000,0.302220,0.833048,0.985700,0.978098,0.779732,0.992356,0.798343
2,1.0,0.798835,1.470639,-1.635975,0.453773,0.425629,1.104875,1.282322,1.381664,0.000000,...,1.128848,0.900461,0.000000,0.909753,1.108330,0.985692,0.951331,0.803252,0.865924,0.780118
3,0.0,1.344385,-0.876626,0.935913,1.992050,0.882454,1.786066,-1.646778,-0.942383,0.000000,...,-0.678379,-1.360356,0.000000,0.946652,1.028704,0.998656,0.728281,0.869200,1.026736,0.957904
4,1.0,1.105009,0.321356,1.522401,0.882808,-1.205349,0.681466,-1.070464,-0.921871,0.000000,...,-0.373566,0.113041,0.000000,0.755856,1.361057,0.986610,0.838085,1.133295,0.872245,0.808487


In [52]:
y = dask_df['label']
X = dask_df[dask_df.columns.difference(['label'])]

In [53]:
cluster = LocalCUDACluster(n_workers=2)

distributed.preloading - INFO - Import preload module: dask_cuda.initialize
distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [54]:
client = Client(cluster)

In [55]:
dtrain = xgb.dask.DaskDMatrix(client, X, y)

In [56]:
output = xgb.dask.train(client,
                            # Use GPU training algorithm
                            {'verbosity':1, 'tree_method': 'gpu_hist', 
                             'objective':'binary:logistic'},
                            dtrain,
                            num_boost_round=10)

[15:05:44] task [xgboost.dask]:tcp://192.168.1.200:45313 got new rank 0
[15:05:44] task [xgboost.dask]:tcp://192.168.1.200:46119 got new rank 1
[15:05:49] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:05:49] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [67]:
booster = output['booster']  # booster is the trained model
booster.set_param({'predictor': 'gpu_predictor'})

predictions = xgb.dask.predict(client, booster, dtrain)

In [68]:
prediction = xgb.dask.inplace_predict(client, output, X)

In [69]:
prediction

Dask Series Structure:
npartitions=87
    float32
        ...
     ...   
        ...
        ...
Name: 0, dtype: float32
Dask Name: getitem, 348 tasks

In [70]:
predictions.compute()

array([0.72380555, 0.81788135, 0.89107114, ..., 0.22989058, 0.39494425,
       0.46599638], dtype=float32)